In [1]:
#U.S. Department of Commerce - 미국 상무부에서 발행하는 연방 관보 고지
#2022년
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from time import sleep

options= Options()
options.headless = True
driver = webdriver.Chrome(options = options)
url= "https://bis.doc.gov/index.php/regulations/federal-register-notices/17-regulations/1372-federal-register-notices-2018"

driver.get(url)
sleep(1)


In [2]:
soup = BeautifulSoup(driver.page_source, 'lxml')
div = soup.find("div", attrs={"itemprop":"articleBody"})

In [3]:
#html 유형 나누기 - 총 23개 중 1번 - 7개, 2번 - 16개
table = div.find("table", attrs={"border":"1"}).find_all("tr", attrs={"valign":"top"})
print("전체 유형 : ", len(table))
type1 = div.find_all("div", attrs={"class":"OutlineElement Ltr SCXW30029558 BCX8"})
print("1번 유형 : ",len(type1))
print("2번 유형 : ", len(table) - len(type1))

전체 유형 :  23
1번 유형 :  0
2번 유형 :  23


In [4]:
#dates 리스트에 date 담기
dates=[]

#1번 유형
for i in range(len(type1)):
    dates.append(table[i].find("span").get_text().strip())

#2번 유형
for i in range(len(type1),len(table)):
    dates.append(table[i].get_text()[1:10].strip())
    
print(len(dates))

23


In [5]:
#titles 리스트에 title 담기
titles=[]

#1번 유형
for i in range(len(table)):
    title = table[i].find("a").get_text() 
    titles.append(title)

print(len(titles))

23


In [6]:
#links 리스트에 link 담기
links = []
for i in range(len(table)):
    link = url + table[i].find("a")['href']
    links.append(link)
print(len(links))

23


In [7]:
#contents 리스트에 content 담기
contents=[]
p_tag = div.find_all("p")
for i in range(4, len(p_tag)):
    if p_tag[i].get_text() and len(p_tag[i].get_text())>1:
        print(p_tag[i].get_text())

 Revisions, Clarifications, and Technical Corrections to the Export Administration Regulations; Correction
                             
03/08/19
84 FR 8485
Normal 0 false false false EN-US X-NONE X-NONE Request for Public Comments Regarding Review of Commerce Control List for Items Transferred from United States Munitions List Categories IV and XV 
Normal 0 false false false EN-US X-NONE X-NONE As part of its work with the National Space Council, the Bureau of Industry and Security, Department of Commerce requests public comment to inform its review of the controls implemented in recent revisions to Categories IV and XV of the United States Munitions List (USML) and the related transfer of items to the Department of Commerce’s Commerce Control List (CCL).  These items include launch vehicles, guided missiles, ballistic missiles, rockets, torpedoes, bombs, and mines; and spacecraft and related articles.  BIS’s review seeks to ensure that the CCL describes these items clearly, captures 

In [15]:
max = len(div.find_all("hr"))
cnt = len(div.find_all("hr"))
start = div.find("hr")
contents = []
while cnt > 0:
    if cnt == max:
        cnt -= 1
        next = start.next_sibling
        string = next.get_text().strip()
    else:
        if str(next)[:3] == "<hr":
            next = next.next_sibling
            contents.append(string)
            string = ""
            cnt -= 1
        else:
            next = next.next_sibling
            string = string +  next.get_text().strip() + ""


print(len(contents))


25
24


In [11]:
last = div.find_all('hr')[-1]

l = 1
while not None:
    if l == 1:
        l -= 1
        la = last.next_sibling
        string = la.get_text().strip()
    else:
        la = la.next_sibling
        if la == None:
            contents.append(string)
            break
        else:
            string += la.get_text().strip() +""
print(len(contents))

Revisions to the Unverified List (UVL)   
Normal 0 false false false EN-US X-NONE X-NONE
Revisions to the Unverified List (UVL)   
Normal 0 false false false EN-US X-NONE X-NONE
01/08/1883 FR 709Revisions, Clarifications, and Technical Corrections to the Export Administration Regulations; CorrectionIn this final rule, the Bureau of Industry and Security corrects an error in the text of Export Control Classification Numbers (ECCNs) 0D606, 0E606, and 8A609.BIS Rule
26


In [14]:
#csv file
import pandas as pd

df = pd.DataFrame()
df['date'] = dates
df['title'] = titles
df['link'] = links
df['content'] = contents
df

ValueError: Length of values (24) does not match length of index (23)

In [ ]:
df.to_csv('미국_상무부_2018년.csv', mode='w', encoding='utf-8-sig')